In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
'''
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784')
'''

"\nfrom sklearn.datasets import fetch_openml\n\nmnist = fetch_openml('mnist_784')\n"

In [ ]:
'''
from sklearn.preprocessing import StandardScaler
mnist_data, mnist_target = mnist.data.values, mnist.target.values
X, y = StandardScaler().fit_transform(mnist_data), mnist_target.astype(np.int)
# X, y = mnist_data, mnist_target.astype(np.int)
y_ = np.zeros((y.size, 10))
y_[np.arange(y.size), y] = 1
'''

'\nfrom sklearn.preprocessing import StandardScaler\nmnist_data, mnist_target = mnist.data.values, mnist.target.values\nX, y = StandardScaler().fit_transform(mnist_data), mnist_target.astype(np.int)\n# X, y = mnist_data, mnist_target.astype(np.int)\ny_ = np.zeros((y.size, 10))\ny_[np.arange(y.size), y] = 1\n'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My\ Drive/Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


In [ ]:
import cv2

input_size = (70, 70)
def get_data(batch_size=25):
  targets = np.random.choice(train_filenames.shape[0], batch_size)
  y_batch = np.zeros(batch_size)
  # X_batch = np.zeros((batch_size, input_size[0], input_size[1], 3))
  X_batch = []
  for i, f in np.ndenumerate(train_filenames[targets]):
    category = f.split('.')[0]
    if category == 'dog':
      y_batch[i] = 0
    else :
      y_batch[i] = 1
    img = cv2.imread(train_path+"/"+f)
    resized_img = cv2.resize(img, dsize=input_size, interpolation=cv2.INTER_CUBIC)
    X_batch.append(resized_img)
  y_batch = y_batch.astype(np.int)
  y_batch_ = np.zeros((y_batch.size, 2))
  y_batch_[np.arange(y_batch.size), y_batch] = 1.
  # return np.array(X_batch).transpose(0,3,1,2) / 255., y_batch_
  return np.array(X_batch).transpose(0,3,1,2), y_batch_

In [ ]:
import os
ㅍtrain_path = './../ml-data/Cat-Dog/train'
train_filenames = np.array(os.listdir(train_path))

In [ ]:
X_test, y_test = get_data(batch_size=200)

## Implementing all layers of CNN totally

### image to col & col to image function

In [ ]:
def im2col(input_data, FH, FW, stride=1, pad=0):
  """
  input_data : (N, C, H, W)
  fh, fw : int
  stride, pad : int
  """
  N, C, H, W = input_data.shape
  OH = (H + 2 * pad - FH) // stride + 1
  OW = (W + 2 * pad - FW) // stride + 1
  img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
  col = np.zeros((N, OH, OW, C, FH, FW))

  for h in range(OH):
    ii = h * stride
    for w in range(OW):
      jj = w * stride
      col[:, h, w, :, :, :] = img[:, :, ii:ii+FH, jj:jj+FW]
  col = col.reshape(N*OH*OW, -1)
  return col

def col2im(col, input_shape, FH, FW, stride=1, pad=0):
  N, C, H, W = input_shape
  OH = (H + 2 * pad - FH) // stride + 1
  OW = (W + 2 * pad - FW) // stride + 1
  col = col.reshape(N, OH, OW, C, FH, FW)
  img = np.zeros((N, C, H+2*pad, W+2*pad))
  for h in range(OH):
    ii = h * stride
    for w in range(OW):
      jj = w * stride
      img[:,:,ii:ii+FH, jj:jj+FW] = col[:, h, w, :, :, :]
  return img[:, :, pad:pad+H, pad:pad+W]

In [ ]:
def col2im1(col, input_shape, FH, FW, stride=1, pad=0):
  N, C, H, W = input_shape
  OH = (H + 2 * pad - FH) // stride + 1
  OW = (W + 2 * pad - FW) // stride + 1
  col = col.reshape(N, OH, OW, C, FH, FW)
  img = np.zeros((N, C, H+2*pad, W+2*pad))
  for h in range(OH):
    ii = h * stride
    for w in range(OW):
      jj = w * stride
      img[:,:,ii:ii+FH, jj:jj+FW] = col[:, h, w, :, :, :]
  return img[:, :, pad:pad+H, pad:pad+W]

def col2im2(col, input_shape, FH, FW, stride=1, pad=0):
  N, C, H, W = input_shape
  OH = (H + 2 * pad - FH) // stride + 1
  OW = (W + 2 * pad - FW) // stride + 1
  col = col.reshape(N, OH, OW, C, FH, FW)
  img = np.zeros((N, C, H+2*pad, W+2*pad))
  for h in range(OH):
    ii = h * stride
    for w in range(OW):
      jj = w * stride
      img[:,:,ii:ii+FH, jj:jj+FW] += col[:, h, w, :, :, :]
  return img[:, :, pad:pad+H, pad:pad+W]

### Activation and Evaluating function

In [ ]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

class Relu:
  def __init__(self):
    self.mask = None

  def forward(self, X):
    self.mask = (X <= 0)
    self.out = X.copy()
    self.out[self.mask] = 0
    return self.out
  
  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout
    return dx

def softmax(x):
  C = np.max(x)
  exps = np.exp(x - C)
  if x.ndim == 1:
    sum_exps = np.sum(exps)
    return exps / sum_exps
  sum_exps = np.sum(exps, axis=1)
  return (exps.T / sum_exps).T

def cross_entropy_error(y, t):
  if y.ndim == 1:
    y = y.reshape(1, y.size)
    t = t.reshape(1, t.size)
  batch_size = t.shape[0]
  return -np.sum( t * np.log(y + 1e-4) ) / batch_size

### Affine function

In [ ]:
class Affine:
  def __init__(self, W, b):
    self.W = W
    self.b = b
    self.x = None
    self.original_x_shape = None
    self.dW = None
    self.db = None

  def forward(self, x):
    self.original_x_shape = x.shape
    self.x = x.reshape(x.shape[0], -1)
    return np.dot(self.x, self.W) + self.b
  
  def backward(self, dout):
    self.db = np.sum(dout, axis=0)
    self.dw = np.dot(self.x.T, dout)
    dx = np.dot(dout, self.W.T)
    return dx.reshape(self.original_x_shape)

### SoftmaxWithLoss function

In [ ]:
class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None
    self.y = None
    self.t = None
  
  def forward(self, x, t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y, self.t)
    return self.loss
  
  def backward(self, dout=1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size
    return dx

### Initor function

In [ ]:
def gaussianInit(input_size, output_size, weight_init_std=0.01):
  return np.random.randn(input_size, output_size) * weight_init_std

def xavierInit(input_size, output_size, weight_init_std=0.01):
  return np.random.randn(input_size, output_size) / np.sqrt(input_size)

def heInit(input_size, output_size, weight_init_std=0.01):
  return np.random.randn(input_size, output_size) * np.sqrt( 2 / input_size)

### Convolution layer

In [ ]:
class Convolution:
  def __init__(self, W, b, stride=1, pad=0):
    self.W = W
    self.b = b
    self.stride = stride
    self.pad = pad

  def forward(self, x):
    self.x = x
    N, C, H, W = self.x.shape
    FN, C, FH, FW = self.W.shape
    out_h = (H + 2*self.pad - FH) // self.stride + 1
    out_w = (W + 2*self.pad - FW) // self.stride + 1
    
    self.col = im2col(x, FH, FW, self.stride, self.pad)
    self.col_W = self.W.reshape(FN, -1).T
    out = np.dot(self.col, self.col_W) + self.b

    out = out.reshape(N, out_h, out_w, FN).transpose(0,3,1,2)
    return out
  
  def backward(self, dout):
    N, C, H, W = self.x.shape
    FN, C, FH, FW = self.W.shape
    out_h = (H + 2*self.pad - FH) // self.stride + 1
    out_w = (W + 2*self.pad - FW) // self.stride + 1
    # step1
    dout = dout.transpose(0,2,3,1).reshape(-1, FN)
    # step2 for db
    self.db = np.sum(dout, axis=0)
    # step3 for self.col * self.col_W
    dcol = np.dot(dout, self.col_W.T)
    dcol_W = np.dot(self.col.T, dout)
    # step4 for dx, dw
    dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)
    self.dw = dcol_W.T.reshape(FN, C, FH, FW)
    return dx

### Pooling layer

In [ ]:
class Pooling:
  def __init__(self, pool_h, pool_w, stride=1, pad=0):
    self.pool_h = pool_h
    self.pool_w = pool_w
    self.stride = stride
    self.pad = pad
  
  def forward(self, x):
    N, C, H, W = x.shape
    out_h = (H - self.pool_h) // self.stride + 1
    out_w = (W - self.pool_w) // self.stride + 1
    col = im2col(x, self.pool_h, self.pool_w, stride=self.stride, pad=self.pad)
    col = col.reshape(-1, self.pool_h * self.pool_w)
    self.arg_max = np.argmax(col, axis=1)
    out = np.max(col, axis=1)
    out = out.reshape(N, out_h, out_w, C).transpose(0,3,1,2)
    self.x = x
    return out # out.shape : N, C, out_h, out_w

  def backward(self, dout): # dout.shape : N, C, out_h, out_w
    dout = dout.transpose(0,2,3,1)  # dout.shape : N, out_h, out_w, C
    pool_size = self.pool_h * self.pool_w
    dmax = np.zeros((dout.size, pool_size))                       # dmax.shape : N x out_h x out_w x C, pool_h x pool_w
    dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()   
    dmax = dmax.reshape(dout.shape + (pool_size,))                # dmax.shape : N, out_h, out_w, C, pool_h x pool_w
    dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)            # dmax.shape : N x out_h x out_w, C x pool_h x pool_w
    dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)  # dx.shape : N, C, H, W
    return dx

In [ ]:
p = Pooling(2, 2, 2, 0)
N, C, H, W = 10, 3, 10, 10
x = np.arange(3000).reshape(N, C, H, W)
out = p.forward(x)

In [ ]:
out.shape

(10, 3, 5, 5)

### Making CNN architecture

<Architecture>
input -- --> Conv -> ReLU -> Pooling -- --> Affine -> ReLU -- --> Affine -> Softmax

In [ ]:
from collections import OrderedDict

class SimpleConvNet:
  # def __init__(self, input_dim=(1, 28, 28), conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1}, \
  #              hidden_size=100, output_size=10, initor="Gaussian", weight_init_std=0.01):
  #  filter_num = conv_param['filter_num']
  #  filter_size = conv_param['filter_size']
  #  filter_pad = conv_param['pad']
  #  filter_stride = conv_param['stride']
  #  input_size = input_dim[1]
  #  conv_output_size = (input_size + 2 * filter_pad - filter_size) // filter_stride + 1
  #  pool_output_size = int(filter_num * (conv_output_size / 2)**2)  # pooling layer's size: 2, stride:2
  def __init__(self, initor='Gaussian', weight_init_std=0.01): 
    conv_output_size = 64
    pool_output_size = int(30 * (conv_output_size / 2) ** 2)
    print(pool_output_size)
    hidden_size = 100
    output_size = 2
    init_functions = {'Gaussin':gaussianInit, 'Xavier':xavierInit, 'He':heInit}
    self.params = {}
    # for Convolution layer 1
    self.params['W1'] = np.random.randn(30, 3, 3, 3) * weight_init_std
    self.params['b1'] = np.zeros(30)
    # for Convolution layer 2
    self.params['W2'] = np.random.randn(30, 30, 3, 3) * weight_init_std
    self.params['b2'] = np.zeros(30)
    # for Convolution layer 3
    self.params['W5'] = np.random.randn(30, 30, 3, 3) * weight_init_std
    self.params['b5'] = np.zeros(30)
    # for Affine layer 1
    self.params['W3'] = init_functions[initor](pool_output_size, hidden_size, weight_init_std)
    self.params['b3'] = np.zeros(hidden_size)
    # for Affine layer 2
    self.params['W4'] = init_functions[initor](hidden_size, output_size, weight_init_std)
    self.params['b4'] = np.zeros(output_size)

    self.layers = OrderedDict()
    self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], stride=1, pad=0)
    self.layers['Relu1'] = Relu()
    self.layers['Conv2'] = Convolution(self.params['W2'], self.params['b2'], stride=1, pad=0)
    self.layers['Relu2'] = Relu()
    self.layers['Conv3'] = Convolution(self.params['W5'], self.params['b5'], stride=1, pad=0)
    self.layers['Relu3'] = Relu()
    self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
    self.layers['Affine1'] = Affine(self.params['W3'], self.params['b3'])
    self.layers['Relu4'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W4'], self.params['b4'])
    self.last_layer = SoftmaxWithLoss()
  
  def predict(self, x):
    for layer in self.layers.values():
      x = layer.forward(x)
    return x
  
  def loss(self, x, t):
    y = self.predict(x)
    return self.last_layer.forward(y, t)
  
  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    if t.ndim != 1 : t = np.argmax(t, axis=1)
    accuracy = np.sum(y == t) / float(x.shape[0])
    return accuracy

  def gradient(self, x, t):
    self.loss(x, t)

    dout = 1
    dout = self.last_layer.backward(dout)
    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
      dout = layer.backward(dout)
    
    grads = {}
    grads['W1'] = self.layers['Conv1'].dw
    grads['b1'] = self.layers['Conv1'].db
    grads['W2'] = self.layers['Conv2'].dw
    grads['b2'] = self.layers['Conv2'].db
    grads['W3'] = self.layers['Affine1'].dw
    grads['b3'] = self.layers['Affine1'].db
    grads['W4'] = self.layers['Affine2'].dw
    grads['b4'] = self.layers['Affine2'].db
    grads['W5'] = self.layers['Conv3'].dw
    grads['b5'] = self.layers['Conv3'].db

    return grads

In [ ]:
X_test.shape, y_test.shape

In [ ]:
net = SimpleConvNet(initor="He")
iter_num = 10000
lr = 0.005
batch_size = 40 
epoch_size = max((iter_num // batch_size), 1)

train_loss_list = []
train_acc_list = []
test_acc_list = []
e1 = 0.9
e2 = 0.1
velocity = {}
for ii in range(1, 5+1):
  velocity['W'+str(ii)] = 0.
  velocity['b'+str(ii)] = 0.
for i in range(iter_num):
  X_batch, y_batch = get_data(batch_size=batch_size) 
  #X_batch, y_batch = X_test, y_test
  grads = net.gradient(X_batch, y_batch)
  for ii in range(1, 5+1):
    velocity['W'+str(ii)] = e1 * velocity['W'+str(ii)] - e2 * grads['W'+str(ii)]
    velocity['b'+str(ii)] = e1 * velocity['b'+str(ii)] - e2 * grads['b'+str(ii)]
    net.params['W'+str(ii)] += velocity['W'+str(ii)]
    net.params['b'+str(ii)] += velocity['b'+str(ii)]
    #net.params['W'+str(ii)] -= lr * grads['W'+str(ii)]
    #net.params['b'+str(ii)] -= lr * grads['b'+str(ii)]
  #loss = net.loss(X_batch, t_batch)
  #train_loss_list.append(loss)

  #if i % epoch_size == 0:
  if i % 10 == 0:
    #train_acc = net.accuracy(X_traiㅑn, y_train)
    test_acc = net.accuracy(X_test, y_test)
    #train_acc_list.append(train_acc)
    #test_acc_list.append(test_acc)
    #print(train_acc, test_acc)
    print(test_acc)

In [ ]:
test_target = np.random.choice(X_test.shape[0], 5)
X_target = X_test[test_target]

In [ ]:
preds = net.predict(X_target)

In [ ]:
cats = preds[:,0] < preds[:,1]

In [ ]:
plt.figure(figsize=(10, 2))
for i in range(5):
  plt.subplot(1,5,i+1)
  print(X_target[i].shape)
  if cats[i] == True:
    plt.title("CAT")
  else:
    plt.title("DOG")
  plt.imshow(X_target[i].transpose(1,2,0))